In [1]:
import datetime
import os
import re

import great_expectations as ge
import numpy as np
import pandas as pd
import rad_pipeline.rad_pipeline as rp
import rad_pipeline.zipcodes as zc
import rad_pipeline.summarize as rs

In [2]:
import importlib
importlib.reload(rp)
importlib.reload(zc)
importlib.reload(ge)
importlib.reload(rs)

<module 'rad_pipeline.summarize' from '/Users/alexhasha/repos/massenergize/rad_pipeline/rad_pipeline/summarize.py'>

In [3]:
zipcode_mapping = pd.read_excel(rp.RAW_DATA_FILES["zip_code_community"], sheet_name="Villages to Muni with Zip")

In [5]:
zipcode_mapping.columns

Index(['Zip Code', 'City', 'Municipality', 'County', 'Unnamed: 4',
       'Unnamed: 5'],
      dtype='object')

In [6]:
zipcode_mapping = zipcode_mapping[["Zip Code", "City", "Municipality", "County"]]

In [4]:
VALID_MUNIS = pd.read_excel(rp.RAW_DATA_FILES["zip_code_community"], sheet_name="351 Mass Munis")

In [24]:
def get_relation(df, col1, col2):        
    first_max = df[[col1, col2]].groupby(col1).nunique().max()[0]
    second_max = df[[col1, col2]].groupby(col2).nunique().max()[0]
    if first_max==1:
        if second_max==1:
            return 'one-to-one'
        elif second_max > 1:
            return 'many-to-one'
    elif first_max > 1:
        if second_max==1:
            return 'one-to-many'
        elif second_max > 1:
            return 'many-to-many'

from itertools import product
for col_i, col_j in product(zipcode_mapping.columns, zipcode_mapping.columns):
    if col_i == col_j:
        continue
    print(f"The relation of {col_i} to {col_j} is {get_relation(zipcode_mapping, col_i, col_j)}")

The relation of Zip Code to City is many-to-many
The relation of Zip Code to Municipality is many-to-many
The relation of Zip Code to County is many-to-one
The relation of City to Zip Code is many-to-many
The relation of City to Municipality is many-to-one
The relation of City to County is many-to-one
The relation of Municipality to Zip Code is many-to-many
The relation of Municipality to City is one-to-many
The relation of Municipality to County is many-to-many
The relation of County to Zip Code is one-to-many
The relation of County to City is one-to-many
The relation of County to Municipality is many-to-many


In [28]:
zipcode_mapping.groupby("Municipality")["County"].nunique().sort_values()

Municipality
Abington       1
Plainville     1
Plainfield     1
Pittsfield     1
Phillipston    1
              ..
Granby         1
Hamilton       1
Yarmouth       1
Nantucket      2
Mansfield      2
Name: County, Length: 350, dtype: int64